In [1]:
import numpy
import sys
from nltk.tokenize import RegexpTokenizer, sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

In [1]:
import pickle
with open('poetry_corpus.pkl', 'rb') as f:
    corpus = pickle.load(f)

In [3]:
poems_in_lines = [poem.splitlines() for poem in corpus]

In [4]:
corpus_in_lines = [line for poem in poems_in_lines for line in poem]

In [5]:
corpus_in_lines[0]

'Dear Writers, I’m compiling the first in what I hope is a series of publications I’m calling artists among artists. The theme for issue 1 is “Faggot Dinosaur.” I hope to hear from you! Thank you and best wishes.'

In [ ]:
import string, re
from nltk.corpus import words
import multiprocessing as mp
# def clean_text(text):
#     if len(sent_tokenize(text)) > 1:
#         return ''
#     text = text.lower()
#     text = text.replace('\%','')
#     text = re.sub('\[.*?\]', '', text)
#     text = re.sub('https?://\S+|www\.\S+', '', text)
#     text = re.sub('<.*?>+', '', text)
#     text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
#     text = re.sub('\n', ' ', text)
#     text = re.sub('\w*\d\w*', '', text)
#     text = " ".join(filter(lambda x:x[0]!="@", text.split()))
#     return text

def clean_text(text):
    cont_dict = { 
        "ain't": "am not / are not / is not / has not / have not",
        "aren't": "are not / am not",
        "can't": "cannot",
        "can't've": "cannot have",
        "'cause": "because",
        "could've": "could have",
        "couldn't": "could not",
        "couldn't've": "could not have",
        "didn't": "did not",
        "doesn't": "does not",
        "don't": "do not",
        "hadn't": "had not",
        "hadn't've": "had not have",
        "hasn't": "has not",
        "haven't": "have not",
        "he'd": "he had / he would",
        "he'd've": "he would have",
        "he'll": "he shall / he will",
        "he'll've": "he shall have / he will have",
        "he's": "he has / he is",
        "how'd": "how did",
        "how'd'y": "how do you",
        "how'll": "how will",
        "how's": "how has / how is / how does",
        "I'd": "I had / I would",
        "I'd've": "I would have",
        "I'll": "I shall / I will",
        "I'll've": "I shall have / I will have",
        "I'm": "I am",
        "I've": "I have",
        "isn't": "is not",
        "it'd": "it had / it would",
        "it'd've": "it would have",
        "it'll": "it shall / it will",
        "it'll've": "it shall have / it will have",
        "it's": "it has / it is",
        "let's": "let us",
        "ma'am": "madam",
        "mayn't": "may not",
        "might've": "might have",
        "mightn't": "might not",
        "mightn't've": "might not have",
        "must've": "must have",
        "mustn't": "must not",
        "mustn't've": "must not have",
        "needn't": "need not",
        "needn't've": "need not have",
        "o'clock": "of the clock",
        "oughtn't": "ought not",
        "oughtn't've": "ought not have",
        "shan't": "shall not",
        "sha'n't": "shall not",
        "shan't've": "shall not have",
        "she'd": "she had / she would",
        "she'd've": "she would have",
        "she'll": "she shall / she will",
        "she'll've": "she shall have / she will have",
        "she's": "she has / she is",
        "should've": "should have",
        "shouldn't": "should not",
        "shouldn't've": "should not have",
        "so've": "so have",
        "so's": "so as / so is",
        "that'd": "that would / that had",
        "that'd've": "that would have",
        "that's": "that has / that is",
        "there'd": "there had / there would",
        "there'd've": "there would have",
        "there's": "there has / there is",
        "they'd": "they had / they would",
        "they'd've": "they would have",
        "they'll": "they shall / they will",
        "they'll've": "they shall have / they will have",
        "they're": "they are",
        "they've": "they have",
        "to've": "to have",
        "wasn't": "was not",
        "we'd": "we had / we would",
        "we'd've": "we would have",
        "we'll": "we will",
        "we'll've": "we will have",
        "we're": "we are",
        "we've": "we have",
        "weren't": "were not",
        "what'll": "what shall / what will",
        "what'll've": "what shall have / what will have",
        "what're": "what are",
        "what's": "what has / what is",
        "what've": "what have",
        "when's": "when has / when is",
        "when've": "when have",
        "where'd": "where did",
        "where's": "where has / where is",
        "where've": "where have",
        "who'll": "who shall / who will",
        "who'll've": "who shall have / who will have",
        "who's": "who has / who is",
        "who've": "who have",
        "why's": "why has / why is",
        "why've": "why have",
        "will've": "will have",
        "won't": "will not",
        "won't've": "will not have",
        "would've": "would have",
        "wouldn't": "would not",
        "wouldn't've": "would not have",
        "y'all": "you all",
        "y'all'd": "you all would",
        "y'all'd've": "you all would have",
        "y'all're": "you all are",
        "y'all've": "you all have",
        "you'd": "you had / you would",
        "you'd've": "you would have",
        "you'll": "you shall / you will",
        "you'll've": "you shall have / you will have",
        "you're": "you are",
        "you've": "you have"
        }

    stopwords_list = stopwords.words("english")
    try:
        while text.startswith('b'):
            text = text[1:]
        text = text.lower()
        for word in cont_dict.keys():
            pattern = re.compile(f'({str(word)})',re.I)
            text = pattern.sub(cont_dict[word],text)
        tokens = word_tokenize(text)
        text = ' '.join([word for word in tokens if word not in stopwords_list])
        text = ' '.join([''.join([c if c.isalnum() else ' ' for c in word]) for word in word_tokenize(text)])
        text = re.sub(' +', ' ', text)
        text = text.replace('\%','')
        text = re.sub('\[.*?\]', '', text)
        text = re.sub('https?://\S+|www\.\S+', '', text)
        text = re.sub('<.*?>+', '', text)
        text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
        text = re.sub('\n', ' ', text)
        text = re.sub('\w*\d\w*', '', text)
        text = " ".join(filter(lambda x:x[0]!="@", text.split()))
        return text.strip()
    
    except AttributeError:
        text = str(text)
        return preprocess_text(text)
    
    except TypeError:
        text = text.decode()[1:]
        return preprocess(text.strip())

with mp.Pool(processes=mp.cpu_count()-1) as pool:
    clean_corpus = pool.map(clean_text, corpus_in_lines)
# p = Process(target=clean_text, args=)
# clean_corpus = Parallel()(delayed(clean_text)(line) for line in corpus_in_lines)
# clean_corpus = list(map(clean_text, corpus_in_lines))

In [ ]:
import pickle
with open('clean_corpus.pkl', 'wb') as f:
    pickle.dump(clean_corpus, f)

In [ ]:
clean_corpus_lines = [line for line in clean_corpus if len(word_tokenize(line)) > 0]

In [27]:
print(clean_corpus_lines[0])

i think that i shall never fear


In [29]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

tokenizer = Tokenizer(num_words=5)
tokenizer.fit_on_texts(clean_corpus_lines)
# input_sequences = tokenizer.texts_to_sequences(corpus_lines)
total_words = len(tokenizer.word_index) + 1
print(total_words)
input_sequences = []
for line in corpus_lines:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

139400


In [37]:
import pandas as pd
word_df = pd.DataFrame.from_dict(tokenizer.word_index, orient='index', columns=['count'])

In [39]:
print(word_df.head())
print(word_df.tail())

     count
the      1
and      2
of       3
a        4
to       5
                 count
protoplanetary  139395
moogoogaipan    139396
nottooold       139397
gooing          139398
morninga        139399
